In [ ]:
# Welcome to your new notebook
# Needed to automatically authenticate to Storage for DuckDB

%pip install msfabricutils -q

In [ ]:
# Install Semantic Link Labs
%pip install semantic-link-labs



In [ ]:
# Install Semantic Link
import pandas as pd
import sempy.fabric as fabric
from datetime import datetime,date,timedelta

!pip install semantic-link -q

# Test import to confirm functionality
try:
    import sempy.fabric as fabric
    print("Successfully imported sempy.fabric - ready to use!")
except ImportError as e:
    print(f"Failed to import sempy.fabric. Error: {str(e)}")

#Update with names for my Workspace and SemanticModelName
WorkspaceName = "PPU Space Testing"
SemanticModelName = "WWI Sales - Azure SQL Source - PPU - 48 Months - 2 Days"

In [ ]:
# Run DAX Query against Semantic Model
import sempy_labs as labs
from sempy_labs.tom import connect_semantic_model
import sempy.fabric as fabric

# DAX query to execute
dax_query = """
EVALUATE
'Customer'
"""

# Connect to the semantic model
with connect_semantic_model(dataset=SemanticModelName, readonly=True, workspace=WorkspaceName) as tom:
    # Execute the DAX query using sempy.fabric
    df_result = fabric.evaluate_dax(dataset=SemanticModelName, dax_string=dax_query, workspace=WorkspaceName)
    
# Display the result
display(df_result)

In [ ]:
# Clean column names
df_result.columns = df_result.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
df_result.columns = df_result.columns.str.replace(' ', '_')
df_result.columns = df_result.columns.str.replace('(', '')
df_result.columns = df_result.columns.str.replace(')', '')

# Changing Columns to DateTime
df_result['Valid_From'] = pd.to_datetime(df_result["Valid_From"], utc=True)
df_result['Valid_To'] = pd.to_datetime(df_result["Valid_To"], utc=True)

# Create a cutoff date (1900-01-01)
cutoff_date = pd.to_datetime(pd.Timestamp('1900-01-01'), utc=True)

# Replace dates before 1900-01-01 with the cutoff date
df_result['Valid_To'] = df_result['Valid_To'].apply(lambda x: cutoff_date if x < cutoff_date else x)

# Validate Changes 
display(df_result)

In [ ]:
# Write to Customers Table

import duckdb
from deltalake import write_deltalake
import pandas as pd
import notebookutils  # Fabric-specific utility
from datetime import datetime

# Define the Table Name for the Lakehouse
table_name = "Customers"

# Define Data Frame Name
dataframe_name = "df_result"

# Define the Table Mode Overwrite existing table; use "append" to add data
table_mode = "overwrite"

# Get the access token for Fabric authentication
access_token = notebookutils.credentials.getToken('storage')

# Define storage options for Fabric Lakehouse
storage_options = {
    "bearer_token": access_token,
    "use_fabric_endpoint": "true",
    "allow_unsafe_rename": "true"  # Useful for overwriting tables
}

# Define the path to the Lakehouse table (adjust to your workspace and lakehouse)
table_path = f"/lakehouse/default/Tables/{table_name}"

# Use DuckDB to query the DataFrame (optional SQL transformation step)
# Here, we just select all data, but you could add complex SQL logic
duckdb_result = duckdb.sql(f"SELECT * FROM {dataframe_name}").arrow()

# Write the result to a Delta table in the Lakehouse
write_deltalake(
    table_path,
    duckdb_result,
    mode=table_mode,  # Overwrite existing table; use "append" to add data
    engine="rust",     # Use the Rust engine for Delta writing
    storage_options=storage_options
)

print(f"Data successfully written to Lakehouse table! {table_path}")